In [1]:
# REQUIRED LIBRARIES
from chapter_5 import Strategy, Engine
import yfinance as yf
import math
 
# BASIC PARAMETERS
TICKERS = ['AMZN','NVDA','AAPL','MSFT','GOOGL']
START_DATE = '2024-01-01'
END_DATE = '2024-12-31'
 
#STRATEGY DEFINITION
class RebalanceExposure(Strategy):    
    def on_bar(self):
        if self.current_idx.weekday() == 0:
            self._rebalance()
            
    def _rebalance(self):
        target_exposure = 0.7 / len(self.tickers)
        total_aum = (sum(
            self.position_size(ticker) 
            *self.close(ticker)
            for ticker in self.tickers)
            + self.cash)
        
        for ticker in self.tickers:
            current_exposure = ((
                self.position_size(ticker)
                * self.close(ticker)) 
                / total_aum)
            delta = (
                target_exposure - current_exposure
                ) * total_aum
            order_size = math.floor(delta / self.close(ticker))
            self._create_order(ticker, order_size)
         
    def _create_order(self, ticker, order_size):
        if order_size > 0:
            self.buy(ticker, order_size)
        elif order_size < 0:
            self.sell(ticker, abs(order_size))
 
# CREATE AND RUN INSTANCE
e = Engine()
for ticker in TICKERS:     
    data = yf.Ticker(ticker).history(
        start=START_DATE,
        end=END_DATE,
        interval='1d')
    e.add_data(data, ticker)
 
e.add_strategy(RebalanceExposure())
out = e.run()    

print('Ann. Returns [%]: ',round(out['annualized_returns'],2))
print('Ann. Volatility [%]: ',round(out['volatility'],2))
print('Sharpe Ratio: ',round(out['sharpe_ratio'],2))
print('Max Drawdown [%]: ',round(out['max_drawdown'],2))
print('Exposure [%]: ',out['exposure'])
print('Trade Journal: ')
out['trade_journal']

100%|██████████| 251/251 [00:00<00:00, 2151.52it/s]


Ann. Returns [%]:  38.41
Ann. Volatility [%]:  15.46
Sharpe Ratio:  2.11
Max Drawdown [%]:  -12.43
Exposure [%]:  68.02771814544666
Trade Journal: 


,type,ticker,entry_date,exit_date,entry_price,exit_price,size,return_pct,pnl,total_fees,market_impact_cost,holding_time
0,long,AAPL,2024-01-09 00:00:00-05:00,2024-04-16 00:00:00-04:00,182.582920,170.718957,3,-6.497849,-35.591888,0.0,0.0,97 days 23:00:00
1,long,AAPL,2024-01-09 00:00:00-05:00,2024-04-30 00:00:00-04:00,182.582920,172.289486,2,-5.637676,-20.586869,0.0,0.0,111 days 23:00:00
2,long,AAPL,2024-01-09 00:00:00-05:00,2024-05-07 00:00:00-04:00,182.582920,182.348708,2,-0.128277,-0.468425,0.0,0.0,118 days 23:00:00
3,long,AAPL,2024-01-09 00:00:00-05:00,2024-05-14 00:00:00-04:00,182.582920,186.637140,2,2.220481,8.108439,0.0,0.0,125 days 23:00:00
4,long,AAPL,2024-01-09 00:00:00-05:00,2024-05-21 00:00:00-04:00,182.582920,190.200491,1,4.172116,7.617571,0.0,0.0,132 days 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
186,long,NVDA,2024-11-05 00:00:00-05:00,2024-12-30 00:00:00-05:00,137.427886,137.477356,3,0.035997,0.148410,0.0,0.0,55 days 00:00:00
187,long,NVDA,2024-11-19 00:00:00-05:00,2024-12-30 00:00:00-05:00,141.297272,137.477356,3,-2.703460,-11.459747,0.0,0.0,41 days 00:00:00
188,long,NVDA,2024-11-26 00:00:00-05:00,2024-12-30 00:00:00-05:00,137.677853,137.477356,3,-0.145628,-0.601491,0.0,0.0,34 days 00:00:00
189,long,NVDA,2024-12-10 00:00:00-05:00,2024-12-30 00:00:00-05:00,138.997212,137.477356,3,-1.093443,-4.559567,0.0,0.0,20 days 00:00:00
